Market Basket Anakysis demo  
70k observations, VzW data set on 01.01.18

In [1]:
##############################################
# VzW Example: Market Basket Analysis        #
# Alvaro Muir, It Analytics Data Engineering #
# Feb 15 2018                                #
##############################################

SRC_DIR = "../data"
DATASET = "data-01-01-18"
src     = ''.join([SRC_DIR,'/', DATASET,'-65k','.csv.bz2'])
LIMIT   = 70000

In [2]:
df = pd.read_csv(src, compression='bz2', nrows=LIMIT)
print("Total rows including the header: {}".format(df.shape[0] + 1))
df.head()

Total rows including the header: 65036


,a.cust_id,a.invc_num,a.invc_dt,a.sls_dist_chnl_type_cd,c.device_prod_nm,a.trans_type_desc,a.ia_item_class_cd,ia_item_category_cd,a.item_cost_amt,a.sales_qty,a.item_cd,a.eqp_desc,b.item_desc,a.offer_nm,a.discount_cd,a.gross_discount,a.fin_upg_flag,a.curr_contract_term,a.cntrct_tenure_sum
0,1.012498e+09,591764,2017-12-31,M,NaN,RETENTION,PREPAID CARDS,E-PIN CARDS,0.00,1,PREPAY REPLENISH,Prepaid Cards,SUREPAY PREPAY REPLINISHMENTS,NaN,NO DISCOUNT,0.00,N,Not_Applicable,i 25 Mos Plus Past
1,1.052492e+09,187606,2017-12-31,M,NaN,RETENTION,PREPAID CARDS,E-PIN CARDS,0.00,1,PREPAY REPLENISH,Prepaid Cards,SUREPAY PREPAY REPLINISHMENTS,NaN,NO DISCOUNT,0.00,N,Not_Applicable,i 25 Mos Plus Past
2,1.053225e+09,60966,2017-12-31,M,iPhone 6S Plus 32GB Silver,ACQUISITION,PREPAID CARDS,BILL PAY KIOSK,0.00,1,PPCREP,Prepaid Cards,BPK REPLENISHMENT OF PREPAID CARDS,NaN,NO DISCOUNT,0.00,N,Not_Applicable,i 25 Mos Plus Past
3,1.056751e+09,370652,2017-12-31,M,MOTO E4,RETENTION,PREPAID CARDS,E-PIN CARDS,0.00,1,PREPAY REPLENISH,Prepaid Cards,SUREPAY PREPAY REPLINISHMENTS,NaN,NO DISCOUNT,0.00,N,Not_Applicable,i 25 Mos Plus Past
4,2.244133e+08,261551,2017-12-31,M,IPHONE 8 PLUS SILVER 64GB VZ,RETENTION,DATA ACCESSORY,ACCESSORY,20.98,1,MOPHPS62GRY,Portable Power,PPW MOPHIE POWERSTATION 6000 SPACE GREY,BUY ANY 3 CAS CLA CHG PPW SCR 89 99 ACC BUNDLE,WITH DISCOUNT,14.99,N,Not_Applicable,i 25 Mos Plus Past


In [3]:
##
#  General cleanup

# remove NaN columns
df = df.dropna(axis=1, how='all')

# remove date entry columns
df = df.drop([col for col in [col for col in df.columns if col.endswith('_dt' or '_rev' or '_step')]], axis=1)
    
# normalize strings
# todo, reafactor this
for c in df.columns:
    if df[c].dtype == 'object':
        df[c] = df[c].str.lower()
        
# replace nulls
df = df.fillna(0)

# remove any columns that have less than 2 unique responses
for(c) in df.columns:
    if len(df[c].unique()) < 2:
        df = df.drop(c, axis=1)

In [4]:
##
# Data specific


# clean up headers, remove alias-dot
df.columns = [col.replace('a.','') for col in df.columns.tolist()]
df.columns = [col.replace('b.','') for col in df.columns.tolist()]
df.columns = [col.replace('c.','') for col in df.columns.tolist()]


# convert cust id's and invoice numbers to strings
df['cust_id'] = df['cust_id'].astype('str').map(lambda x: x.rstrip('0').rstrip('.'))
df['invc_num'] = df['invc_num'].astype('str')


# remove rows that contain tablets or IP devices
to_remove = ['apple tablet', 'android tablet', 'windows tablet',
             'verizon wireless home phone','windows tablet',
             'ip virtual devices']

df = df[~df['eqp_desc'].isin(to_remove)]

df.head()

,cust_id,invc_num,sls_dist_chnl_type_cd,device_prod_nm,trans_type_desc,ia_item_class_cd,ia_item_category_cd,item_cost_amt,sales_qty,item_cd,eqp_desc,item_desc,offer_nm,discount_cd,gross_discount,fin_upg_flag,curr_contract_term,cntrct_tenure_sum
0,1012498217,591764,m,0,retention,prepaid cards,e-pin cards,0.00,1,prepay replenish,prepaid cards,surepay prepay replinishments,0,no discount,0.00,n,not_applicable,i 25 mos plus past
1,1052492345,187606,m,0,retention,prepaid cards,e-pin cards,0.00,1,prepay replenish,prepaid cards,surepay prepay replinishments,0,no discount,0.00,n,not_applicable,i 25 mos plus past
2,1053225113,60966,m,iphone 6s plus 32gb silver,acquisition,prepaid cards,bill pay kiosk,0.00,1,ppcrep,prepaid cards,bpk replenishment of prepaid cards,0,no discount,0.00,n,not_applicable,i 25 mos plus past
3,1056751445,370652,m,moto e4,retention,prepaid cards,e-pin cards,0.00,1,prepay replenish,prepaid cards,surepay prepay replinishments,0,no discount,0.00,n,not_applicable,i 25 mos plus past
4,224413346,261551,m,iphone 8 plus silver 64gb vz,retention,data accessory,accessory,20.98,1,mophps62gry,portable power,ppw mophie powerstation 6000 space grey,buy any 3 cas cla chg ppw scr 89 99 acc bundle,with discount,14.99,n,not_applicable,i 25 mos plus past


In [5]:
# types of accessories
df['eqp_desc'].unique()

array(['prepaid cards', 'portable power', 'display protector', 'case',
       'vhcl power chg', 'bluetooth', 'gift cards', 'jetpack devices',
       'accessory - other', 'basic', 'smart accessory',
       'smartwatch wearables', 'charger', 'automotive devices',
       'speakers', 'combo pack', 'memory card', 'headset',
       'non embedded sim', 'smartwatch accessory', 'data interface',
       'keyboard', 'dock', 'battery', 'other tablet',
       'non smartwatch wearable devices', 'other connected devices',
       'usb / pc card devices', 'antenna', 'cantenna devices',
       'hf router', 'network extenders', 'ip desk phones'], dtype=object)

In [6]:
# let's get a view of distinct counts

purchases = df[['cust_id','invc_num', 'device_prod_nm', 'item_desc', 'sales_qty']]
purchases.dropna(axis=0, subset=['invc_num'], inplace=True)

customers = purchases.cust_id.unique()
invoices = purchases.invc_num.unique()
devices  = purchases.device_prod_nm.unique()
items = [' '.join(i.split()) for i in purchases.item_desc.unique()] # there's weird spaces in there

print('unique counts')
pd.DataFrame(data={'customers': [len(customers)], 
                   'invoices': [len(invoices)], 
                   'devices': [len(devices)], 
                   'items': [len(items)]})


unique counts


,customers,devices,invoices,items
0,28984,618,34005,1308


In [7]:
# lets create a 'basket' for inspection

basket = (purchases[['invc_num', 'item_desc', 'sales_qty']]
          .groupby(['invc_num', 'item_desc'])['sales_qty']
          .sum().unstack().reset_index().fillna(0)
          .set_index('invc_num'))

basket.head()

item_desc,$20 account replenishment,arlo go,bat lg transpyre std rep battery,bat lg v10 std replace battery,bat ms735 battery,bat nov bat vzw jetpack mifi 6620,bat sam galaxyj3 replacement battery,blu apple airpods,blu beats solo3 wireless onear hph black,blu beats solo3 wireless onear hph gold,...,ubi sam sfemto 4g network extender,verizon ellipsis jetpack mhs900l,verizon hum x,verizon jetpack 4g lte mhs mifi 7730l,vz jetpack 4g lte mobile hotspot- ac791l,vzw100hgccar0717-$100 hanging gc carrier,vzwcushgccar0717-custom value gc carrier,vzwgccar0915: gift card carrier,vzwposgccar0717-pos evergreen gc carrier,wireless handset for w60
invc_num,,,,,,,,,,,,,,,,,,,,,
1000368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000373,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000376,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000381,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# 1-hot encoding (Could be done in sk-learn...)
basket_sets = basket.applymap(lambda x: 0 if x <= 0 else 1)
basket_sets.head()

item_desc,$20 account replenishment,arlo go,bat lg transpyre std rep battery,bat lg v10 std replace battery,bat ms735 battery,bat nov bat vzw jetpack mifi 6620,bat sam galaxyj3 replacement battery,blu apple airpods,blu beats solo3 wireless onear hph black,blu beats solo3 wireless onear hph gold,...,ubi sam sfemto 4g network extender,verizon ellipsis jetpack mhs900l,verizon hum x,verizon jetpack 4g lte mhs mifi 7730l,vz jetpack 4g lte mobile hotspot- ac791l,vzw100hgccar0717-$100 hanging gc carrier,vzwcushgccar0717-custom value gc carrier,vzwgccar0915: gift card carrier,vzwposgccar0717-pos evergreen gc carrier,wireless handset for w60
invc_num,,,,,,,,,,,,,,,,,,,,,
1000368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000373,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000376,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1000381,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)
frequent_itemsets.head()

,support,itemsets
0,0.023144,[blu apple airpods]
1,0.011175,[bpk replenishment of prepaid cards]
2,0.010234,[cas belkin trifld samsungtab e 8 black]
3,0.011263,[cas lifeproof fre gals8 asphalt]
4,0.036348,[cas ob defender gals8 black]


In [10]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules

,antecedants,consequents,support,confidence,lift
0,(cas ob defender gals8 black),(cla vzw usbc pd20 vpc),0.036348,0.487864,3.973609
1,(cla vzw usbc pd20 vpc),(cas ob defender gals8 black),0.122776,0.144431,3.973609
2,(scr zagg invisishield glass gal8 curved),(cas ob defender gals8 black),0.143861,0.188062,5.173991
3,(cas ob defender gals8 black),(scr zagg invisishield glass gal8 curved),0.036348,0.744337,5.173991
4,(cas ob symmetry gals8 black),(scr zagg invisishield glass gal8 curved),0.018586,0.653481,4.542441
5,(scr zagg invisishield glass gal8 curved),(cas ob symmetry gals8 black),0.143861,0.084424,4.542441
6,(scr zagg invisishield glass gal8 curved),(chg vzw usbc pd20 tvl),0.143861,0.089330,2.712188
7,(chg vzw usbc pd20 tvl),(scr zagg invisishield glass gal8 curved),0.032936,0.390179,2.712188
8,(scr zagg invisishield glass gal8 curved),(cla vzw usbc pd20 vpc),0.143861,0.477719,3.890976
9,(cla vzw usbc pd20 vpc),(scr zagg invisishield glass gal8 curved),0.122776,0.559760,3.890976


In [11]:
# gonna change this with larger dataset
rules[ (rules['lift'] >= 1) &
       (rules['confidence'] >= 0.6) ]

,antecedants,consequents,support,confidence,lift
3,(cas ob defender gals8 black),(scr zagg invisishield glass gal8 curved),0.036348,0.744337,5.173991
4,(cas ob symmetry gals8 black),(scr zagg invisishield glass gal8 curved),0.018586,0.653481,4.542441
16,"(cas ob defender gals8 black, cla vzw usbc pd2...",(scr zagg invisishield glass gal8 curved),0.017733,0.845771,5.879078


In [12]:
basket['scr zagg invisishield glass gal8 curved'].sum()

5416.0